<a href="https://colab.research.google.com/github/WoradeeKongthong/association_rule_learning/blob/master/01_Covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note :  
The dataset I use in this learning is part of https://www.kaggle.com/bitsofishan/covid19-patient-symptoms ' dataset.  
I choose only the record that the corona result is 1.

In [221]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
columns = [1,2,3,4,5,7,8,9,10,12,13,14,15]

In [142]:
columns = [i for i in range(16) if ((i!=6)&(i!=11))]

columns

[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15]

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/WoradeeKongthong/association_rule_learning/master/corona_symptoms.csv',usecols=[i for i in range(1,16) if ((i!=6)&(i!=11))])

# Dataset : covid19 symptoms

In [146]:
df

,sour throat,weakness,breathing problem,drowsiness,pain in chest,diabetes,heart disease,lung disease,stroke or reduced immunity,high blood pressue,kidney disease,change in appetide,Loss of sense of smell
0,1,1,1,1,1,0,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0,0,0,0,0,1,0
2,1,0,0,1,0,1,1,0,1,0,1,0,0
3,1,1,0,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0,0,0,1,0,0,0,0,0,0,1,0,0
65,1,1,1,1,1,1,0,0,1,0,0,1,1
66,0,0,0,1,1,1,0,0,0,0,0,1,1
67,0,0,0,1,1,1,0,0,0,1,0,1,0


# Data Proprocessor

input of a priori model is a list of lists of symptoms in each case

In [0]:
symptoms = []
for i in range(len(df)):
  temp = df.iloc[i,:]
  symptoms.append(list(temp[temp==1].index.values))

In [148]:
# example of symptoms list
symptoms[:5]

[['sour throat',
  'weakness',
  'breathing problem',
  'drowsiness',
  'pain in chest',
  'change in appetide'],
 ['weakness', 'change in appetide'],
 ['sour throat',
  'drowsiness',
  'diabetes',
  'heart disease',
  'stroke or reduced immunity',
  'kidney disease'],
 ['sour throat', 'weakness', 'change in appetide'],
 ['weakness', 'pain in chest', 'kidney disease', 'change in appetide']]

# Association Rule Learning

Simple learning of Covid-19 symptoms that occur together.

In [0]:
from apyori import apriori
rules = apriori(symptoms, min_support=0.0145, min_confidence=0.01, min_lift=0, min_length=2)

Note  
- consider every symptoms, even one that occur in one case  
    min_support = 1/69 = 0.0145
- let's set the minimum likelihood that the symptom B is occured if symptom A is occur to very low value (0.01) to capture all possibilities
- let's consider the likelihood that symptoms A&B are occured is at least 0 times more than the likelihood of only symptoms B is occured, to see the symptoms that are unlikely to occur together
- min_length is the minimum number of item in the rules

# Results of the learning

In [0]:
results = list(rules)

In [215]:
len(results)

770

let's see the first 10 rules

In [0]:
results1 = []
for i in range(len(results)):
  items = list(results[i].items)
  support = results[i].support
  confidence = results[i].ordered_statistics[0][2]
  lift = results[i].ordered_statistics[0][3]

  row = (items, support, confidence, lift)
  results1.append(row)

labels = ['rule','support','confidence','lift']
result_df = pd.DataFrame(results1, columns=labels)

In [220]:
result_df.sort_values('lift', ascending=False)

,rule,support,confidence,lift
202,"[drowsiness, kidney disease, heart disease]",0.028986,1.000000,9.857143
437,"[stroke or reduced immunity, drowsiness, kidne...",0.028986,1.000000,5.307692
726,"[breathing problem, change in appetide, sour t...",0.028986,1.000000,5.307692
649,"[breathing problem, diabetes, pain in chest, L...",0.028986,1.000000,5.307692
651,"[breathing problem, sour throat, diabetes, Los...",0.028986,1.000000,5.307692
...,...,...,...,...
207,"[drowsiness, sour throat, high blood pressue]",0.028986,0.250000,0.466216
82,"[Loss of sense of smell, breathing problem, dr...",0.057971,0.235294,0.450980
371,"[pain in chest, breathing problem, sour throat...",0.028986,0.222222,0.414414
134,"[breathing problem, diabetes, sour throat]",0.028986,0.222222,0.414414


# Discussion

Top 10 rules that occur in the record

In [265]:
result_df.sort_values('support', ascending=False)[:10]

,rule,support,confidence,lift
10,[sour throat],0.536232,0.536232,1.000000
4,[drowsiness],0.521739,0.521739,1.000000
12,[weakness],0.463768,0.463768,1.000000
1,[breathing problem],0.449275,0.449275,1.000000
0,[Loss of sense of smell],0.434783,0.434783,1.000000
9,[pain in chest],0.420290,0.420290,1.000000
2,[change in appetide],0.405797,0.405797,1.000000
35,"[change in appetide, drowsiness]",0.318841,0.785714,1.505952
55,"[drowsiness, sour throat]",0.318841,0.611111,1.139640
6,[high blood pressue],0.289855,0.289855,1.000000


Note : the symptoms that occur in more than 50% of the record are 'sour throat' and 'drowsiness'

In [244]:
result_df[(result_df['confidence']==1)]

,rule,support,confidence,lift
159,"[change in appetide, drowsiness, diabetes]",0.101449,1.0,1.916667
174,"[pain in chest, change in appetide, lung disease]",0.028986,1.0,2.379310
175,"[change in appetide, weakness, lung disease]",0.028986,1.0,2.156250
194,"[diabetes, stroke or reduced immunity, lung di...",0.043478,1.0,5.307692
202,"[drowsiness, kidney disease, heart disease]",0.028986,1.0,9.857143
...,...,...,...,...
765,"[breathing problem, change in appetide, sour t...",0.028986,1.0,2.156250
766,"[breathing problem, sour throat, diabetes, pai...",0.028986,1.0,2.156250
767,"[change in appetide, sour throat, diabetes, pa...",0.028986,1.0,2.156250
768,"[breathing problem, change in appetide, sour t...",0.028986,1.0,2.156250


Note : confidence = 1 means all symptoms in the rule always occur together.
There are 254 rules that always occur together

In [218]:
result_df[result_df['lift']==1]

,rule,support,confidence,lift
0,[Loss of sense of smell],0.434783,0.434783,1.0
1,[breathing problem],0.449275,0.449275,1.0
2,[change in appetide],0.405797,0.405797,1.0
3,[diabetes],0.246377,0.246377,1.0
4,[drowsiness],0.521739,0.521739,1.0
5,[heart disease],0.072464,0.072464,1.0
6,[high blood pressue],0.289855,0.289855,1.0
7,[kidney disease],0.101449,0.101449,1.0
8,[lung disease],0.130435,0.130435,1.0
9,[pain in chest],0.420290,0.420290,1.0


Note : when lift = 1, it means there is no association between item(s). I got only single item rule in this case.

In [246]:
result_df.sort_values('lift', ascending=False)[:5]

,rule,support,confidence,lift
202,"[drowsiness, kidney disease, heart disease]",0.028986,1.0,9.857143
437,"[stroke or reduced immunity, drowsiness, kidne...",0.028986,1.0,5.307692
726,"[breathing problem, change in appetide, sour t...",0.028986,1.0,5.307692
649,"[breathing problem, diabetes, pain in chest, L...",0.028986,1.0,5.307692
651,"[breathing problem, sour throat, diabetes, Los...",0.028986,1.0,5.307692


The maximum lift items : drowsiness, kidney disease, and heart disease  
support = 0.029 means that 2.9% of the records (or 2 out of 69) contains  drowsiness and kidney disease.  
confidence = 1 means that 100% of the records (2 records in this case) also have heart disease.  
lift = 9.857 means that the likelihood of drowsiness, kidney disease, and heart disease occur together is 9.857 times more than only heart disease.

In [0]:
# function to investigate list of symptoms

def symptoms_checker(symptoms):
  symptom_occur_list = np.ones((len(result_df),), dtype=bool)

  for symptom in symptoms :
    temp_array = np.array([symptom in items for items in result_df['rule']])
    symptom_occur_list = symptom_occur_list & temp_array
    
  return result_df[symptom_occur_list].sort_values('confidence', ascending=False)

Let's look at the records contain 'heart disease'.

In [318]:
symptom_df = symptoms_checker(['heart disease'])
symptom_df

,rule,support,confidence,lift
204,"[drowsiness, stroke or reduced immunity, heart...",0.028986,1.000000,5.307692
203,"[drowsiness, sour throat, heart disease]",0.028986,1.000000,1.864865
443,"[stroke or reduced immunity, kidney disease, s...",0.028986,1.000000,5.307692
434,"[drowsiness, stroke or reduced immunity, sour ...",0.028986,1.000000,5.307692
433,"[stroke or reduced immunity, drowsiness, kidne...",0.028986,1.000000,5.307692
432,"[drowsiness, kidney disease, sour throat, hear...",0.028986,1.000000,1.864865
370,"[pain in chest, breathing problem, sour throat...",0.028986,1.000000,1.864865
222,"[pain in chest, sour throat, heart disease]",0.028986,1.000000,1.864865
221,"[stroke or reduced immunity, kidney disease, h...",0.028986,1.000000,5.307692
220,"[kidney disease, sour throat, heart disease]",0.028986,1.000000,1.864865


consider 'heart disease' and 'drowsiness'

In [319]:
symptom_df = symptoms_checker(['heart disease','drowsiness'])
symptom_df

,rule,support,confidence,lift
202,"[drowsiness, kidney disease, heart disease]",0.028986,1.000000,9.857143
203,"[drowsiness, sour throat, heart disease]",0.028986,1.000000,1.864865
204,"[drowsiness, stroke or reduced immunity, heart...",0.028986,1.000000,5.307692
432,"[drowsiness, kidney disease, sour throat, hear...",0.028986,1.000000,1.864865
433,"[stroke or reduced immunity, drowsiness, kidne...",0.028986,1.000000,5.307692
434,"[drowsiness, stroke or reduced immunity, sour ...",0.028986,1.000000,5.307692
620,"[sour throat, kidney disease, heart disease, d...",0.028986,1.000000,5.307692
50,"[drowsiness, heart disease]",0.028986,0.055556,0.766667
